#### Import the required libraries

In [ ]:
import psycopg2 as psy
import pandas as pd
import json
import os
import time

#### Establish a connection with the database

In [ ]:
database_host = "localhost"
database_name = "dbl_data_challenge"
database_user = "admin"
database_pass = "vZtbqKNXGz27cQCH"

In [ ]:
con = psy.connect(
    host = database_host,
    database = database_name,
    user = database_user,
    password = database_pass
)
cur = con.cursor()

#### Create a table for the replies

In [ ]:
cur.execute("DROP TABLE IF EXISTS replies")

In [ ]:
cur.execute("""
    CREATE TABLE replies
        (
            id_str TEXT DEFAULT NULL,
            in_reply_to_status_id_str TEXT DEFAULT NULL
        )
""")

#### Loads a given data array with values into the database

In [ ]:
def insert_into_database(data): 
    cur.execute("""INSERT INTO replies (
        id_str,
        in_reply_to_status_id_str
    )
    VALUES (%s, %s)""", (data[0],data[1]))
    con.commit()

#### This cell uses the same technique as with the extraction of the conversations to extract reply pairs

In [ ]:
cur.execute("""
    SELECT id_str, in_reply_to_status_id_str FROM tweets
""")

convo_pairs = {}
reverse_pairs = {}

data = cur.fetchall()

for row in data: 
    convo_pairs[row[0]] = row[1]

for row in data: 
    if row[1] in reverse_pairs: 
        reverse_pairs[row[1]].append(row[0])
    else: 
        reverse_pairs[row[1]] = [row[0]]
    
del reverse_pairs[None]

start = time.time()

for idx, id_str in enumerate(reverse_pairs): 
    for reply in reverse_pairs[id_str]: 
        data = [
            id_str,
            reply
        ]
        insert_into_database(data)
    if (idx+1) % 10000 == 0: 
        end = time.time()
        print("Retrieved replies for "+str(idx+1)+" tweets in "+str(end-start)+" seconds")
        start = time.time()

In [ ]:
cur.close()
con.close()